#### Setup

In [ ]:
from pathlib import Path

import numpy as np
import scipy.sparse as sparse
import torch
import torch.nn as nn
from PIL import Image
from sklearn.decomposition import PCA
from torchvision import models, transforms
from IPython.display import clear_output

In [ ]:
%cd "/content/drive/My Drive/archive/imecc/texture/data"

/content/drive/My Drive/archive/imecc/texture/data


#### Utils

In [ ]:
def get_files(dataset):
    """Get files from a dataset folder."""
    dataset = Path(dataset)
    files = [sorted(dataset.rglob(ext)) 
           for ext in ["*.png", "*.jpg", "*.bmp", "*.ppm"]]
    files = [file for ext in files for file in ext]
    return files

def zscore(arr):
    """Measure an array z-score."""
    return (arr - arr.mean(0)) / (arr.std(0) + 10 ** -8)

#### Feature Extraction: Backes

In [ ]:
class Backes2013():
    """Complex network image descriptor generator.
    From "Texture analysis and classification:
    A complex network-based approach by Backes et al."
    """

    def gen_index(self, r, shape):
        """Generate valid indices for graph modeling."""
        x0 = []
        y0 = []
        k0 = []
        for i in range(shape[0]):
            for j in range(shape[1]):
                x0.append(i)
                y0.append(j)
                k0.append(i * shape[1] + j)
        x1 = []
        y1 = []
        k1 = []
        wd = []
        for i in range(-r, r + 1):
            for j in range(-r, r + 1):
                w = (i ** 2 + j ** 2) / (2 * r ** 2) # DIFF FROM Ribas
                if w > 0 and w <= .5:                # DIFF FROM Ribas
                    x1.append(i)
                    y1.append(j)
                    k1.append(i * shape[1] + j)
                    wd.append(w)
        len0 = len(x0)
        len1 = len(x1)
        x0 = np.repeat(x0, len1)
        y0 = np.repeat(y0, len1)
        k0 = np.repeat(k0, len1)
        x1 = x0 + np.tile(x1, len0)
        y1 = y0 + np.tile(y1, len0)
        k1 = k0 + np.tile(k1, len0)
        wd = np.tile(wd, len0)
        valid = len0 * len1 * [True]
        valid = np.logical_and(valid, x1 >= 0)
        valid = np.logical_and(valid, x1 < shape[0])
        valid = np.logical_and(valid, y1 >= 0)
        valid = np.logical_and(valid, y1 < shape[1])
        x0 = x0[valid]
        y0 = y0[valid]
        k0 = k0[valid]
        x1 = x1[valid]
        y1 = y1[valid]
        k1 = k1[valid]
        wd = wd[valid]
        return x0, y0, k0, x1, y1, k1, wd
  
    def extract(self, dataset, tag, r=3, tstart=.005, tstop=.515, tstep=.015):
        """Extract complex network features 
        from a graph by thresholding its edges.""" 
        filepath = Path(f"./{dataset}_{tag}.npz")
        if not filepath.exists():
            files = get_files(dataset)
            im0 = Image.open(files[0]).convert("L")
            nsteps = int((tstop - tstart) / tstep) + 1
            gr_shape = tuple(2 * [im0.size[0] * im0.size[1]])
            x0, y0, k0, x1, y1, k1, wd = self.gen_index(r, im0.size)
            X = {}
            X["contrast"] = np.zeros((len(files), nsteps))
            X["energy"] = np.zeros((len(files), nsteps))
            X["entropy"] = np.zeros((len(files), nsteps))
            X["mean"] = np.zeros((len(files), nsteps))
            for i, f in enumerate(files):
                img = Image.open(f)
                img = img.convert("L")
                img = img.resize(im0.size)
                img = np.array(img, dtype=float)
                if img.shape != im0.size:
                    img = img.T
                w = wd + (img[x0, y0] - img[x1, y1]) / 510
                graph = sparse.csr_matrix((w, (k0, k1)), gr_shape)
                base_degseq = (graph > 0).sum(axis=0).A
                for j, t in enumerate(np.arange(tstart, tstop, tstep)):
                    degseq = base_degseq - (graph > t).sum(axis=0).A
                    deg, probs = np.unique(degseq, return_counts=True)
                    probs = probs / probs.sum()
                    X["contrast"][i, j] = np.dot(deg * deg, probs)
                    X["energy"][i, j] = np.dot(probs, probs)
                    X["entropy"][i, j] = - np.dot(probs, np.log2(probs))
                    X["mean"][i, j] = np.dot(deg, probs)
                print(f"{dataset} loading {i + 1} / {len(files)}")
                clear_output(wait=True)
            X["contrast"] = zscore(X["contrast"])
            X["energy"] = zscore(X["energy"])
            X["entropy"] = zscore(X["entropy"])
            X["mean"] = zscore(X["mean"])
            np.savez_compressed(filepath, **X)
        print(f"{filepath.stem} extracted")
        clear_output(wait=True)

In [ ]:
# bc = Backes2013()

# bc.extract("brodatz", "brodatz+bc.npz")
# bc.extract("vistex", "vistex+bc.npz")
# bc.extract("outex13i", "outex13i+bc.npz")
# bc.extract("umd", "umd+bc.npz")
# bc.extract("uiuc", "uiuc+bc.npz")
# bc.extract("kthtips2b", "kthtips2b+bc.npz")
# bc.extract("fmd", "fmd+bc.npz")
# bc.extract("dtd", "dtd+bc.npz")

#### Feature Extraction: Ribas

In [ ]:
class Ribas2020:
    """Neural and complex network image descriptor generator.
    From the paper Fusion of complex networks and
    randomized neural networks for texture analysis
    and classification by Ribas et al.
    """

    def gen_index(self, r, shape):
        """Generate valid indices for graph modeling."""
        x0 = []
        y0 = []
        k0 = []
        for i in range(shape[0]):
            for j in range(shape[1]):
                x0.append(i)
                y0.append(j)
                k0.append(i * shape[1] + j)
        x1 = []
        y1 = []
        k1 = []
        wd = []
        for i in range(-r, r + 1):
            for j in range(-r, r + 1):
                w = np.sqrt(i ** 2 + j ** 2)
                if w > 0 and w <= r:
                    x1.append(i)
                    y1.append(j)
                    k1.append(i * shape[1] + j)
                    wd.append(w)
        len0 = len(x0)
        len1 = len(x1)
        x0 = np.repeat(x0, len1)
        y0 = np.repeat(y0, len1)
        k0 = np.repeat(k0, len1)
        x1 = x0 + np.tile(x1, len0)
        y1 = y0 + np.tile(y1, len0)
        k1 = k0 + np.tile(k1, len0)
        wd = np.tile(wd, len0)
        valid = len0 * len1 * [True]
        valid = np.logical_and(valid, x1 >= 0)
        valid = np.logical_and(valid, x1 < shape[0])
        valid = np.logical_and(valid, y1 >= 0)
        valid = np.logical_and(valid, y1 < shape[1])
        x0 = x0[valid]
        y0 = y0[valid]
        k0 = k0[valid]
        x1 = x1[valid]
        y1 = y1[valid]
        k1 = k1[valid]
        wd = wd[valid]
        return x0, y0, k0, x1, y1, k1, wd

    def gen_hidden_weight(self, Q, p):
        """Generate a hidden weight matrix of neurons."""
        E = Q * (p + 1)
        a = E + 2
        b = E + 3
        c = E ** 2
        V = [E + 1]
        for i in range(E - 1):
            V.append((a * V[-1] + b) % c)
        V = np.array(V)
        V = zscore(V)
        V = V.reshape(Q, p + 1)
        return V

    def gen_output_weight(self, W, D, X):
        """Generate output weight array of neurons."""
        Ms = []
        for Xi in X:
            Z = np.tanh(np.dot(W, Xi))
            Z = np.vstack((- np.ones((1, Z.shape[1])), Z))
            M = np.dot(Z, Z.T) + (10 ** -3) * np.eye(Z.shape[0])
            M = np.dot(np.dot(D, Z.T), linalg.inv(M))
            Ms.append(M)
        return np.ravel(Ms)

    def extract(self, dataset, tag, Q=[4, 14, 19], R=[4, 10]):
        """Extract neural and complex network features from a graph."""
        filepath = Path(f"{dataset}_{tag}.npz")
        if not filepath.exists():
            files = get_files(dataset)
            qr = [[q, r] for q in Q for r in R]
            im0 = Image.open(files[0])
            im0 = im0.convert("L")
            img_shape = im0.size
            n_vertices = im0.size[0] * im0.size[1]
            gr_shape = (n_vertices, n_vertices)
            x0, y0, k0, x1, y1, k1, wd = self.gen_index(max(R), img_shape)
            W = {f"Q{q}R{r}": self.gen_hidden_weight(q, r) for q, r in qr}
            X = {f"Q{q}R{r}": np.zeros((len(files), 3 * q + 3)) for q, r in qr}
            for k, f in enumerate(files):
                img = Image.open(f)
                img = img.convert("L")
                img = img.resize(img_shape)
                img = np.array(img, dtype=float)
                if img.shape != img_shape:
                    img = img.T
                D = img.ravel()
                has_edge = img[x0, y0] < img[x1, y1]
                wi = (img[x1, y1] - img[x0, y0]) / 255
                X0 = - np.ones((max(R) + 1, n_vertices))
                X1 = - np.ones((max(R) + 1, n_vertices))
                X2 = - np.ones((max(R) + 1, n_vertices))
                for r in range(1, max(R)):
                    i = np.logical_and(has_edge, wd <= r)
                    w = wi[i] + (wd[i] - 1) / (r - 1 + 10 ** -3)
                    graph = sparse.csr_matrix((w, (k0[i], k1[i])), gr_shape)
                    X0[r, :] = zscore((graph > 0).sum(axis=0).A.ravel())
                    X1[r, :] = zscore(graph.sum(axis=0).A.ravel())
                    X2[r, :] = zscore(graph.sum(axis=1).A.ravel())
                for q, r in qr:
                    Wqr = W[f"Q{q}R{r}"]
                    Xqr = [X0[:r + 1, :], X1[:r + 1, :], X2[:r + 1, :]]
                    X[f"Q{q}R{r}"][k, :] = self.gen_output_weight(Wqr, D, Xqr)
                print(f"{dataset} loading {k + 1} / {len(files)}")
                clear_output(wait=True)
            for key in X:
                X[key] = zscore(X[key])
            np.savez_compressed(f"{dataset}_{tag}", **X)
        print(f"{filepath.stem} extracted")
        clear_output(wait=True)

In [ ]:
# rb = Ribas2020()

# rb.extract("brodatz", "brodatz+rb.npz")
# rb.extract("vistex", "vistex+rb.npz")
# rb.extract("outex13i", "outex13i+rb.npz")
# rb.extract("umd", "umd+rb.npz")
# rb.extract("uiuc", "uiuc+rb.npz")
# rb.extract("kthtips2b", "kthtips2b+rb.npz")
# rb.extract("fmd", "fmd+rb.npz")
# rb.extract("dtd", "dtd+rb.npz")

#### Feature Extraction: CNN
(Remember that ResNet does not work)

In [ ]:
class VGGCutModel(nn.Module):
    """Truncated vgg model without the last 3 layers."""
    def __init__(self, model):
        super(VGGCutModel, self).__init__()
        self.features = model.features
        self.avgpool = model.avgpool
        self.classifier = nn.Sequential(
            *list(model.classifier.children())[:-3])
  
    def forward(self, x):
        """Forward pass of the truncated neural network model."""
        with torch.no_grad():
            x = self.features(x)
            x = self.avgpool(x)
            x = self.classifier(x.view(-1))
            x = x.numpy().reshape(-1, )
        return x

class ResNetCutModel():
    """Truncated resnet model without the last layer."""
    def __init__(self, model):
        super(ResNetCutModel, self).__init__()
        modules = list(model.children())[:-1]
        self.model = nn.Sequential(*modules)
        for p in self.model.parameters():
            p.requires_grad = False

    def forward(self, x):
        """Forward pass of the truncated neural network model."""
        x = self.model.forward(x)
        x = x.numpy().reshape(-1, )
        return x

class NNFeatureExtractor():
    """Neural network feature extractor."""
    def __init__(self, model, n_features):
        self.model = model
        self.n_features = n_features
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
  
    def convert(self, img):
        """Convert PIL image to an array of cnn features"""
        img = img.convert("RGB")
        img = self.transform(img)
        img = img.view([1, 3, 224, 224])
        return self.model.forward(img)
        
    def extract(self, infolder, outfile):
        """Extract CNN features from a image file"""
        outfile = Path(outfile)
        if not outfile.exists():
            files = get_files(infolder)
            x = np.zeros((len(files), self.n_features))
            for i, f in enumerate(files):
                img = Image.open(f)
                x[i, :] = self.convert(img)
                print(f"{infolder} loading {i} / {len(files)}")
                clear_output(wait=True)
            np.savez_compressed(outfile, x=x)
        print(f"{outfile.stem} extracted")
        clear_output(wait=True)

In [ ]:
model = VGGCutModel(models.vgg11(pretrained=True))
cnn = NNFeatureExtractor(model, 4096)

cnn.extract("brodatz", "brodatz+vgg11.npz")
cnn.extract("vistex", "vistex+vgg11.npz")
cnn.extract("outex13i", "outex13i+vgg11.npz")
cnn.extract("umd", "umd+vgg11.npz")
cnn.extract("uiuc", "uiuc+vgg11.npz")
cnn.extract("kthtips2b", "kthtips2b+vgg11.npz")
cnn.extract("fmd", "fmd+vgg11.npz")
cnn.extract("dtd", "dtd+vgg11.npz")

dtd+vgg11 extracted


In [ ]:
model = VGGCutModel(models.vgg19(pretrained=True))
cnn = NNFeatureExtractor(model, 4096)

cnn.extract("brodatz", "brodatz+vgg19.npz")
cnn.extract("vistex", "vistex+vgg19.npz")
cnn.extract("outex13i", "outex13i+vgg19.npz")
cnn.extract("umd", "umd+vgg19.npz")
cnn.extract("uiuc", "uiuc+vgg19.npz")
cnn.extract("kthtips2b", "kthtips2b+vgg19.npz")
cnn.extract("fmd", "fmd+vgg19.npz")
cnn.extract("dtd", "dtd+vgg19.npz")

dtd+vgg19 extracted


#### Feature Extraction: CNN + Visibility Graph

In [ ]:
class VisibilityGraph():
    """Visibility graph generator."""
    def __init__(self):
        self.sight = 0
        self.visibility = None
    
    def get_shape(self, files):
        """Get image and its graph shape."""
        im0 = Image.open(files[0])
        im0 = im0.convert("L")
        img_shape = im0.size
        gr_shape = tuple(2 * [img_shape[0] * img_shape[1]])
        return img_shape, gr_shape
  
    def nvisibility(self, x, left, right, i, graph):
        """Natural visibility graph algorithm."""
        max_slope = float("-inf")
        min_slope = float("+inf")
        for j in np.arange(i + 1, right):
            slope = (x[j] - x[i]) / (j - i)
            if slope > self.sight:
                break
            if slope > max_slope:
                max_slope = slope
                graph[i, j] = 1
                graph[j, i] = 1
        for j in np.arange(i - 1, left, -1):
            slope = (x[i] - x[j]) / (i - j)
            if slope < - self.sight:
                break
            if slope < min_slope:
                min_slope = slope
                graph[i, j] = 1
                graph[j, i] = 1

    def hvisibility(self, x, left, right, i, graph):
        """Horizontal visibility graph algorithm."""
        max_datum = float("-inf")
        for j in np.arange(i + 1, right):
            if x[j] > max_datum:
                max_datum = x[j]
                graph[i, j] = 1
                graph[j, i] = 1
                max_datum = float("-inf")
        for j in np.arange(i - 1, left, -1):
            if x[j] > max_datum:
                max_datum = x[j]
                graph[i, j] = 1
                graph[j, i] = 1

    def wvisibility(self, x, left, right, i, graph):
        """Weighted visibility graph algorithm."""
        max_slope = float("-inf")
        min_slope = float("+inf")
        for j in np.arange(i + 1, right):
            slope = (x[j] - x[i]) / (j - i)
            if slope > self.sight:
                break
            if slope > max_slope:
                max_slope = slope
                graph[i, j] = np.arctan(slope)
                graph[j, i] = np.arctan(slope)
        for j in np.arange(i - 1, left, -1):
            slope = (x[i] - x[j]) / (i - j)
            if slope < - self.sight:
                break
            if slope < min_slope:
                min_slope = slope
                graph[i, j] = np.arctan(slope)
                graph[j, i] = np.arctan(slope)

    def sort_and_conquer(self, x):
        """Sort and conquer algorithm proposed by Ghosh et al."""
        n = x.size
        sortd = np.argsort(x)[::-1]
        graph = sparse.lil_matrix((n, n))
        for i in np.arange(n):
            current = sortd[i]
            connected = graph.rows[current]
            left = -1
            right = n
            for j in connected:
                if j < current:
                    left = max(left, j)
                else:
                    right = min(right, j)
            self.visibility(x, left, right, current, graph)
        return sparse.csr_matrix(graph)

    def extract(self, infile, mode, sight=0):
        """Generate a (horizontal) visibility graph for each feature vector."""
        
        # validation: if infile exists
        infile = Path(infile)
        if not infile.exists():
            print(f"{infile} does not exist")
            clear_output(wait=True)
            return
        
        # validation: if visibility is valid
        if mode == "nvg":
            outfile = f"{infile.stem}+nvg.npz"
            self.visibility = self.nvisibility
        elif mode == "hvg":
            outfile = f"{infile.stem}+hvg.npz"
            self.visibility = self.hvisibility
        elif mode == "wvg":
            outfile = f"{infile.stem}+wvg{sight:05.2f}.npz"
            self.visibility = self.wvisibility
        else:
            print("visibility does not exist")
            clear_output(wait=True)
            return
        
        # validation: if outfile exists
        outfile = Path(outfile)
        if outfile.exists():
            print(f"{outfile.stem} exists")
            clear_output(wait=True)
            return

        # validation: if sight greater or equal than zero
        if self.sight < 0:
            print("sight less than zero")
            clear_output(wait=True)
            return

        # extraction
        x = np.load(infile, allow_pickle=True)["x"]
        degseq = np.zeros_like(x)
        for i in range(x.shape[0]):
            vg = self.sort_and_conquer(x[i, :])
            degseq[i, :] = vg.sum(axis=0).A
        np.savez_compressed(outfile, degseq=degseq)
        print(f"{outfile.stem} extracted")
        clear_output(wait=True)

In [ ]:
vg = VisibilityGraph()

for dataset in ["brodatz", "vistex", "outex13i",
                "umd", "uiuc", "kthtips2b", "fmd", "dtd"]:
    for cnn in ["vgg11", "vgg19"]:
        infile = f"{dataset}+{cnn}.npz"
        vg.extract(infile, "nvg")
        vg.extract(infile, "hvg")
        vg.extract(infile, "wvg", 00.36)
        vg.extract(infile, "wvg", 00.84)
        vg.extract(infile, "wvg", 01.73)
        vg.extract(infile, "wvg", 05.67)

dtd+vgg19+wvg05.67 exists


#### Feature Extraction: Image Visibility Graph

In [ ]:
class Iacovacci2018(VisibilityGraph):
    """Image (horizontal) visibility graph generator as proposed by Iacovacci et al.
    From the paper visibility graphs for image processing by Iacovacci and Lacasa
    """
    def __init__(self):
        super().__init__()

    def gen_patch_profile(self, img, mode="n"):
        """Generate a patch profile z of an image (horizontal) visibility graph."""
        Z = np.zeros(256)
        nvg_motiff = lambda a, b, c: c > 2 * b - a
        hvg_motiff = lambda a, b, c: b < a and b < c
        is_motiff = nvg_motiff if mode == "n" else hvg_motiff
        for i in range(img.shape[0] - 2):
            for j in range(img.shape[1] - 2):
                z  = 0
                z += 128 * is_motiff(img[i + 0, j], img[i + 0, j + 1], img[i + 0, j + 2])
                z += 64  * is_motiff(img[i + 1, j], img[i + 1, j + 1], img[i + 1, j + 2])
                z += 32  * is_motiff(img[i + 2, j], img[i + 2, j + 1], img[i + 2, j + 2])
                z += 16  * is_motiff(img[i, j + 0], img[i + 1, j + 0], img[i + 2, j + 0])
                z += 8   * is_motiff(img[i, j + 1], img[i + 1, j + 1], img[i + 2, j + 1])
                z += 4   * is_motiff(img[i, j + 2], img[i + 1, j + 2], img[i + 2, j + 2])
                z += 2   * is_motiff(img[i, j    ], img[i + 1, j + 1], img[i + 2, j + 2])
                z += 1   * is_motiff(img[i, j + 2], img[i + 1, j + 1], img[i + 2, j    ])
                Z[z] += 1
        return Z

    def gen_degree_distribution(self, img, mode="n"):
        """Generate a degree distribution of a image (horizontal) visibility graph."""
        row = []
        col = []

        # set visibility mode
        if mode == "n":
            self.visibility = self.nvisibility
        elif mode == "h":
            self.visibility = self.hvisibility

        # rows degree distribution
        j = np.arange(img.shape[1])
        for i in range(img.shape[0]):
            graph = self.sort_and_conquer(img[i, j])
            rowi, coli = graph.nonzero()
            ind = i * img.shape[1] + j
            rowi = [ind[k] for k in rowi]
            coli = [ind[k] for k in coli]
            row.extend(rowi)
            col.extend(coli)

        # columns degree distribution
        i = np.arange(img.shape[0])
        for j in range(img.shape[1]):
            graph = self.sort_and_conquer(img[i, j])
            rowi, coli = graph.nonzero()
            ind = i * img.shape[1] + j
            rowi = [ind[k] for k in rowi]
            coli = [ind[k] for k in coli]
            row.extend(rowi)
            col.extend(coli)

        # diagonals degree distribution
        for k in range(-img.shape[0] + 1, img.shape[1]):
            i = [max(0, -k)]
            j = [max(0, +k)]
            while i[-1] + 1 < img.shape[0] and j[-1] + 1 < img.shape[1]:
                i.append(i[-1] + 1)
                j.append(j[-1] + 1)
            i = np.array(i)
            j = np.array(j)
            graph = self.sort_and_conquer(img[i, j])
            rowi, coli = graph.nonzero()
            ind = i * img.shape[1] + j
            rowi = [ind[k] for k in rowi]
            coli = [ind[k] for k in coli]
            row.extend(rowi)
            col.extend(coli)

        # degree distribution extraction
        w = np.ones(len(row))
        gr_shape = (img.size, img.size)
        graph = sparse.csr_matrix((w, (row, col)), gr_shape)
        deg, probs = np.unique(graph.sum(axis=0).A, return_counts=True)
        probs = probs / probs.sum()
        probs = probs.copy()
        probs.resize(30, refcheck=False)
        return probs

    def extract(self, dataset, tag):
        """Extract visibility graph features from specified images."""
        files = get_files(dataset)
        img_shape, _ = self.get_shape(files)
        x = {}
        x["3patch_vg"] = np.zeros((len(files), 256))
        x["3patch_hvg"] = np.zeros((len(files), 256))
        x["degdist_vg"] = np.zeros((len(files), 30))
        x["degdist_hvg"] = np.zeros((len(files), 30))
        for k in range(len(files)):
            img = Image.open(files[k])
            img = img.convert("L")
            img = np.array(img, dtype=float)
            img = img if img.shape == img_shape else img.T
            x["n"] = k + 1
            x["3patch_vg"][k, :] = self.gen_patch_profile(img)
            x["degdist_vg"][k, :] = self.gen_degree_distribution(img)
            x["3patch_hvg"][k, :] = self.gen_patch_profile(img, "h")
            x["degdist_hvg"][k, :] = self.gen_degree_distribution(img, "h")

        for key in x:
            x[key] = zscore(x[key])
        np.savez_compressed(f"./{dataset}_{tag}", **x)
        print(f"{dataset} loading {k + 1} / {len(files)}")
        clear_output(wait=True)

#### ~~Feature Extraction: Blending Features~~

In [ ]:
class Blender:
    def reduce_dimension(self, x, threshold=.9):
        variance = 1
        n_components = 0
        pca = PCA().fit(x)
        while variance > threshold:
            n_components += 1
            variance = pca.explained_variance_ratio_[:-n_components].sum()
        x = pca.transform(x)[:, :n_components]
        return x
  
    # Falta terminar isso!!!
    def extract(self, old_tags, old_features, new_tag):
        datasets = [sorted(Path("./").glob(tag)) for tag in old_tags]
        for dataset in zip(*datasets):
            filepath = dataset[0].stem.split("_")[0]
            filepath = Path(f"{filepath}_{new_tag}.npz")
            if not filepath.exists():
                x = [np.load(datapath)[feature] 
                    for i, datapath in enumerate(dataset) 
                    for feature in old_features[i]]
                x = np.hstack(x)
                x = self.reduce_dimension(x)
                np.savez_compressed(filepath, x=x)
            clear_output()
            print(f"{filepath.stem} extracted")

In [ ]:
blender = Blender()

In [ ]:
# Quais combinações eu analiso?